In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Step 1: Load the merged data from CSV
file_path = '/Users/anhpham/Downloads/FIN2/MIDTERM/merged_data.csv'
merged_data = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')

In [2]:
# Step 2: Calculate log returns for each stock (tickers columns)
def calculate_log_returns(prices):
    return np.log(prices / prices.shift(1))

log_returns = merged_data.drop(columns=['Mkt-RF', 'SMB', 'HML', 'RF']).apply(calculate_log_returns)
print("Log Returns:\n", log_returns.head())

Log Returns:
                                AAPL      NVDA      ORCL      MSFT       WMT  \
Date                                                                          
2019-01-01 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2019-02-01 00:00:00+00:00  0.039524  0.070564  0.041079  0.070250  0.032443   
2019-03-01 00:00:00+00:00  0.096882  0.152900  0.029859  0.055671 -0.014859   
2019-04-01 00:00:00+00:00  0.054901  0.007988  0.029716  0.101963  0.058380   
2019-05-01 00:00:00+00:00 -0.136476 -0.289821 -0.084895 -0.054442 -0.013707   

                               COST        PM       JPM       AXP       VTI  \
Date                                                                          
2019-01-01 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2019-02-01 00:00:00+00:00  0.018968  0.125056  0.016363  0.051993  0.034969   
2019-03-01 00:00:00+00:00  0.104330  0.016540 -0.030451  0.014375  0.008675   
2019-04-01 00:00:00+00:00  0.013903 -

In [3]:
# Step 3: Calculate excess returns (log returns minus RF)
excess_returns = log_returns.sub(merged_data['RF'], axis=0)
print("Excess Returns:\n", excess_returns.head())

Excess Returns:
                                AAPL      NVDA      ORCL      MSFT       WMT  \
Date                                                                          
2019-01-01 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2019-02-01 00:00:00+00:00  0.037724  0.068764  0.039279  0.068450  0.030643   
2019-03-01 00:00:00+00:00  0.094982  0.151000  0.027959  0.053771 -0.016759   
2019-04-01 00:00:00+00:00  0.052801  0.005888  0.027616  0.099863  0.056280   
2019-05-01 00:00:00+00:00 -0.138576 -0.291921 -0.086995 -0.056542 -0.015807   

                               COST        PM       JPM       AXP       VTI  \
Date                                                                          
2019-01-01 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2019-02-01 00:00:00+00:00  0.017168  0.123256  0.014563  0.050193  0.033169   
2019-03-01 00:00:00+00:00  0.102430  0.014640 -0.032351  0.012475  0.006775   
2019-04-01 00:00:00+00:00  0.01180

In [4]:
# Step 4: Add the Fama-French factors (Mkt-RF, SMB, HML) for regression analysis
factors = merged_data[['Mkt-RF', 'SMB', 'HML']]

In [5]:
# Step 5: Estimate betas for each stock using OLS regression
def estimate_betas(stock_returns, factors):
    X = sm.add_constant(factors)  # Add constant to the regression model
    model = sm.OLS(stock_returns, X, missing='drop').fit()  # Fit OLS model
    return model.params[1:]  # Return betas (excluding the intercept)

betas = pd.DataFrame(columns=['Mkt-RF', 'SMB', 'HML'], index=log_returns.columns)

for ticker in log_returns.columns:
    betas.loc[ticker] = estimate_betas(excess_returns[ticker], factors)
print("Betas:\n", betas)

Betas:
          Mkt-RF       SMB       HML
AAPL   1.334304 -0.420244 -0.459603
NVDA   1.742429 -0.204344 -1.076422
ORCL   1.030778 -0.542851  -0.18113
MSFT   0.951279 -0.552991 -0.483952
WMT    0.527688  -0.44859 -0.184422
COST    0.77661 -0.265952 -0.446553
PM     0.686216 -0.415343  0.541431
JPM    1.025311  0.294295  0.794159
AXP    1.163283  0.076799  0.515405
VTI    1.010225 -0.048011  0.043635
IVV    1.003041 -0.190601  0.036296
VOO    1.001328  -0.18803  0.040082
FDFIX  0.978739 -0.169204  0.016858
VFTAX  1.047652 -0.129826 -0.052882
VLACX  1.010702 -0.151508   0.01133
^GSPC   0.98615 -0.174573  0.028698


In [6]:
def calculate_expected_return(betas, risk_free_rate, factors):
    """
    Calculate the expected return of a stock using the Fama-French 3-factor model.
    
    Parameters:
    - betas: pandas DataFrame with columns ['Mkt-RF', 'SMB', 'HML'] and rows for each stock.
    - risk_free_rate: The risk-free rate (e.g., annualized 10-year treasury yield).
    - factors: A DataFrame containing the factor returns for ['Mkt-RF', 'SMB', 'HML'].
    
    Returns:
    - expected_returns: A pandas DataFrame with the expected returns for each stock, indexed by date.
    """
    # Ensure the factors DataFrame is aligned with betas (check for matching stock tickers)
    expected_returns = pd.DataFrame(index=factors.index, columns=betas.index)
    
    for ticker in betas.index:
        if ticker in betas.index:
            expected_returns[ticker] = risk_free_rate + betas.loc[ticker, 'Mkt-RF'] * factors['Mkt-RF'] + \
                                       betas.loc[ticker, 'SMB'] * factors['SMB'] + \
                                       betas.loc[ticker, 'HML'] * factors['HML']
        else:
            # If any ticker is missing from factors, we leave it as NaN
            expected_returns[ticker] = np.nan

    # Drop rows where all expected returns are NaN (if necessary)
    expected_returns = expected_returns.dropna(how='all')
    
    return expected_returns

In [7]:
risk_free_rate = 0.04  # Assume 4% annual risk-free rate for example

# Calculate the expected returns for each stock
expected_returns = calculate_expected_return(betas, risk_free_rate, merged_data[['Mkt-RF', 'SMB', 'HML']])

print(expected_returns)

                               AAPL      NVDA      ORCL      MSFT       WMT  \
Date                                                                          
2019-01-01 00:00:00+00:00  0.142047  0.185323  0.111766  0.106159  0.072236   
2019-02-01 00:00:00+00:00  0.089165  0.124204  0.068772  0.074067  0.053698   
2019-03-01 00:00:00+00:00  0.086430  0.109748  0.075358  0.087269  0.067085   
2019-04-01 00:00:00+00:00  0.090273  0.089438  0.086347  0.076824  0.064681   
2019-05-01 00:00:00+00:00 -0.036203 -0.052736 -0.020132 -0.007305  0.013626   
2019-06-01 00:00:00+00:00  0.134508  0.167713  0.111181  0.107763  0.076604   
2019-07-01 00:00:00+00:00  0.061829  0.059620  0.061892  0.059718  0.054071   
2019-08-01 00:00:00+00:00  0.037634  0.051490  0.035056  0.051855  0.045941   
2019-09-01 00:00:00+00:00  0.032042 -0.005975  0.047743  0.026204  0.039412   
2019-10-01 00:00:00+00:00  0.075000  0.095753  0.063101  0.067188  0.053073   
2019-11-01 00:00:00+00:00  0.097639  0.127433  0.079

In [8]:
# Calculate the Variance-Covariance (VCV) Matrix
vcv_matrix = log_returns.cov()  # Use the covariance method from pandas
print("Variance-Covariance Matrix:")
print(vcv_matrix)

Variance-Covariance Matrix:
           AAPL      NVDA      ORCL      MSFT       WMT      COST        PM  \
AAPL   0.007312  0.007827  0.003672  0.003996  0.001970  0.003281  0.002332   
NVDA   0.007827  0.020203  0.006269  0.005902  0.001904  0.004999  0.001418   
ORCL   0.003672  0.006269  0.005868  0.002821  0.001535  0.002140  0.002007   
MSFT   0.003996  0.005902  0.002821  0.003911  0.001266  0.002142  0.001101   
WMT    0.001970  0.001904  0.001535  0.001266  0.002694  0.002033  0.000665   
COST   0.003281  0.004999  0.002140  0.002142  0.002033  0.004097  0.000378   
PM     0.002332  0.001418  0.002007  0.001101  0.000665  0.000378  0.004420   
JPM    0.002799  0.004268  0.002967  0.001844  0.000731  0.001186  0.002954   
AXP    0.003586  0.004052  0.003109  0.002330  0.001269  0.002021  0.003122   
VTI    0.003753  0.005005  0.002855  0.002543  0.001319  0.002155  0.002054   
IVV    0.003717  0.004912  0.002845  0.002545  0.001342  0.002159  0.002026   
VOO    0.003707  0.00491